In [1]:
import pandas as pd
from sqlalchemy import create_engine


# import yahoo finance
import yfinance as yf
# pip install yfinance

# Put postgreSQL credeitials into sql_login.py
from sql_login import USERNAME as USERNAME
from sql_login import PASSWORD as PASSWORD

## Get data on Chipotle Stock Prie

In [56]:
# Set ticke for Chipotle Mexican Grill, Inc.
#cmg = yf.Ticker("CMG")

# get stock info
#cmg.info

In [2]:
# Retreive historical stock price data
stock_data_df = yf.download("CMG", start="2015-01-01", end="2020-01-10", group_by="ticker")

[*********************100%***********************]  1 of 1 completed


In [3]:
stock_data_df = stock_data_df.reset_index()

In [9]:
stock_data_df.columns = map(str.lower, stock_data_df.columns)

stock_data_df = stock_data_df.rename(columns={"adj close":"adj_close"})

stock_data_df['id'] = [j for j in range(0,stock_data_df.shape[0])]

stock_data_df

,date,open,high,low,close,adj_close,volume,id
0,2015-01-02,686.000000,687.469971,671.010010,678.400024,678.400024,324800,0
1,2015-01-05,678.400024,679.919983,664.200012,667.690002,667.690002,374400,1
2,2015-01-06,666.789978,672.000000,653.770020,664.380005,664.380005,609400,2
3,2015-01-07,668.400024,694.849976,668.400024,694.260010,694.260010,685600,3
4,2015-01-08,702.500000,727.969971,702.500000,719.989990,719.989990,1169900,4
...,...,...,...,...,...,...,...,...
1260,2020-01-06,862.000000,862.799988,854.000000,858.020020,858.020020,405200,1260
1261,2020-01-07,863.000000,863.000000,854.390015,860.080017,860.080017,243300,1261
1262,2020-01-08,856.150024,867.000000,856.150024,856.729980,856.729980,262500,1262
1263,2020-01-09,866.859985,877.000000,859.469971,862.669983,862.669983,457300,1263


In [10]:
rds_connection_string = USERNAME+':'+PASSWORD+"@localhost:5432/avocados"
engine = create_engine(f'postgresql://{rds_connection_string}')
conn = engine.engine.connect()

In [11]:
# Create table to hold yfinance data
conn.execute('DROP TABLE IF EXISTS cmg;')

In [12]:
conn.execute(
"""
    CREATE TABLE cmg (
    id INT PRIMARY KEY,
	date date,
	open DECIMAL(12,7),
	high DECIMAL(12,7),
	low DECIMAL(12,7),
	close DECIMAL(12,7),
    adj_close DECIMAL(12,7),
    volume INT);
"""
)

In [13]:
# Confirm tables
engine.table_names()

['cmg']

## Load DataFrames into database

In [14]:
stock_data_df[['id','date','open','high','low','close','adj_close','volume']].to_sql(name='cmg', con=engine, if_exists='append', index=False)